In [9]:
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf

from tensorflow import keras
from itertools import product
from sklearn.preprocessing import MinMaxScaler

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
with tf.device('/device:CPU:0'):

    # Open velocity data set
    ds = xr.open_dataset('/Users/smata/Downloads/processedData/velocityData.nc')

    # Reformat to Pandas dataframe
    df_u = ds['u'].to_dataframe().reset_index()
    df_v = ds['v'].to_dataframe().reset_index()

    df_u = df_u.pivot(index='time', columns='height', values='u')
    df_v = df_v.pivot(index='time', columns='height', values='v')

    df_u.columns = [f'u_{int(height)}m' for height in df_u.columns]
    df_v.columns = [f'v_{int(height)}m' for height in df_v.columns]

    df = pd.concat([df_u, df_v], axis=1)
    df.reset_index(drop=True, inplace=True)

    df.insert(0, 'L', ds.L.values)
    df.insert(0, 'TKE', ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    windCols = [col for col in df.columns if col.startswith('u_') or col.startswith('v_')]
    stabilityCols = ['TKE', 'L']

    wind_scaler = MinMaxScaler()
    df[windCols] = wind_scaler.fit_transform(df[windCols])

    stability_scaler = MinMaxScaler()
    df[stabilityCols] = stability_scaler.fit_transform(df[stabilityCols])

    # Define and extract input and output columns
    inputs = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 'u_10m', 'v_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    split_index = int(0.8 * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    num_layers_options    = np.arange(1, 10)
    units_options         = [32, 64, 128, 256, 512]
    learning_rate_options = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

    def build_model(num_layers, units, learning_rate):
        model = keras.Sequential()
        
        for _ in range(num_layers):
            model.add(keras.layers.Dense(units = units, activation = 'relu'))
        
        model.add(keras.layers.Dense(y_train.shape[1]))  # Output size should match the number of wind components
        
        model.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
                    loss = 'mean_squared_error',
                    metrics = ['mae'])
        
        return model

    param_grid = list(product(num_layers_options, units_options, learning_rate_options))

    best_model  = None
    best_mae    = float('inf')
    best_params = None

    for num_layers, units, learning_rate in param_grid:
        # Create and train the model
        model = build_model(num_layers, units, learning_rate)
        
        # Train the model (use early stopping or set epochs according to your needs)
        history = model.fit(X_train, y_train, 
                            validation_split = 0.2, 
                            epochs = 100, 
                            batch_size = 32,
                            verbose = 1)  # Set verbose=1 to see training output

        # Evaluate the model on validation data
        val_loss, val_mae = model.evaluate(X_test, y_test, verbose = 0)
        
        # Check if this model is the best
        if val_mae < best_mae:
            best_mae    = val_mae
            best_model  = model
            best_params = (num_layers, units, learning_rate)

    # Print the best hyperparameters
    print("Best MAE:", best_mae)
    print("Best parameters: Layers:", best_params[0], "Units:", best_params[1], "Learning Rate:", best_params[2])

Epoch 1/100
144/144 [==============================] - 0s 675us/step - loss: 0.0246 - mae: 0.1047 - val_loss: 0.0083 - val_mae: 0.0707
Epoch 2/100
144/144 [==============================] - 0s 390us/step - loss: 0.0075 - mae: 0.0655 - val_loss: 0.0068 - val_mae: 0.0636
Epoch 3/100
144/144 [==============================] - 0s 393us/step - loss: 0.0067 - mae: 0.0624 - val_loss: 0.0070 - val_mae: 0.0632
Epoch 4/100
144/144 [==============================] - 0s 410us/step - loss: 0.0068 - mae: 0.0622 - val_loss: 0.0064 - val_mae: 0.0630
Epoch 5/100
144/144 [==============================] - 0s 410us/step - loss: 0.0067 - mae: 0.0620 - val_loss: 0.0062 - val_mae: 0.0607
Epoch 6/100
144/144 [==============================] - 0s 402us/step - loss: 0.0069 - mae: 0.0631 - val_loss: 0.0077 - val_mae: 0.0692
Epoch 7/100
144/144 [==============================] - 0s 389us/step - loss: 0.0069 - mae: 0.0629 - val_loss: 0.0067 - val_mae: 0.0630
Epoch 8/100
144/144 [==============================] - 

KeyboardInterrupt: 